In [1]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection

Here we define the dataset depending on kitti sequence!

In [2]:
DATASET_PATH = os.path.join('/Users/cedric/Datasets/semantic_kitti/')
SEQUENCE_NUM = 7

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM)

ind_start = 0
ind_end = 100
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters


out_folder = 'pcd_preprocessed/'

Now we aggregate a large point cloud based on (ind_start, ind_end)

In [3]:
process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                              major_voxel_size=major_voxel_size,
                              out_folder=out_folder,sequence_num=SEQUENCE_NUM)


PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE


KeyboardInterrupt: 

In [3]:
##load data if already stored 
pcd_ground_minor, pcd_nonground_minor,\
	all_poses, T_pcd, first_position = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size)

Now we subsample the poses based on a voxel_size

In [4]:
poses, positions, \
sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start)

Now we can split the point cloud into chunks based on a tbd chunk_size

In [5]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices, center_positions, \
center_ids, chunk_bounds = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, overlap=overlap, major_voxel_size=major_voxel_size)

Downsampled from (441534, 3) to (7148, 3) points (non-ground)
Downsampled from (210612, 3) to (4546, 3) points (ground)
Downsampled from (328813, 3) to (5497, 3) points (non-ground)
Downsampled from (204410, 3) to (4111, 3) points (ground)


In [9]:
patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)

for sequence in range(len(center_ids)):

    print("Start of sequence ", sequence)

    first_id = patchwise_indices[sequence][0]
    center_id = center_ids[sequence]
    center_position = center_positions[sequence]
    chunk_indices = indices[sequence]

    cam_indices_global, hpr_mask_indices = get_indices_feature_reprojection(sampled_indices_global, first_id, adjacent_frames=(16,13)) 
    
    pcd_chunk = pcd_nonground_chunks[sequence]
    pcd_ground_chunk = pcd_ground_chunks[sequence]
    chunk_major = pcd_nonground_chunks_major_downsampling[sequence]

    points_major = np.asarray(chunk_major.points)
    num_points_major = points_major.shape[0]   

    print(num_points_major, "points in downsampled chunk (major)")

    #tarl_features = tarl_features_per_patch(dataset, chunk_major, center_id, T_pcd, center_position, sampled_indices_global, chunk_size, major_voxel_size)

    cams = ["cam2", "cam3"]

    sam_features_minor, chunk_minor = image_based_features_per_patch(dataset, pcd_nonground_minor, chunk_indices, T_pcd, cam_indices_global, cams, cam_id=0, hpr_radius=2000, dino=False, rm_perp=0.0)
    #point2dino
    #dinov2_features_minor = dinov2_mean(point2dino)
    
    sam_features_major = -1 * np.ones((num_points_major, sam_features_minor.shape[1]))
    #dinov2_features_major = np.zeros((num_points_major, dinov2_features_minor.shape[1])) 

    sam_features_major = kDTree_1NN_feature_reprojection(sam_features_major, chunk_major, sam_features_minor, chunk_minor)
    #dinov2_features_major = kDTree_1NN_feature_reprojection(dinov2_features_major, chunk_major, dinov2_features_minor, chunk_minor)

    zero_rows = np.sum(~np.array(sam_features_major).any(1))
    ratio = zero_rows / num_points_major

    if ratio > 0.3:
        print("The ratio of points without image-based features is", ratio, ". Skipping this chunk.")
        continue

    spatial_distance = cdist(points_major, points_major)
    #dinov2_distance = cdist(dinov2_features_major, dinov2_features_major)
    #tarl_distance = cdist(tarl_features, tarl_features)

    proximity_threshold = 1 # meters that points can be apart from each other and still be considered neighbors
    alpha = 6.0 # weight of the spatial proximity term 
    beta = 0.0 # weight of the label similarity term
    gamma = 0.0 # weight of the dinov2 feature similarity term
    theta = 3.0 # weight of the tarl feature similarity term

    sam_edge_weights, mask = sam_label_distance(sam_features_major, spatial_distance, proximity_threshold, beta)
    #spatial_edge_weights = mask * np.exp(-alpha * spatial_distance)
    #dinov2_edge_weights = mask * np.exp(-gamma * dinov2_distance)
    #tarl_edge_weights = mask * np.exp(-theta * tarl_distance)

    A = sam_edge_weights #spatial_edge_weights * sam_edge_weights * dinov2_edge_weights * tarl_edge_weights
    print("Adjacency Matrix built")

    # Remove isolated points
    chunk_major, A = remove_isolated_points(chunk_major, A)
    print(num_points_major - np.asarray(chunk_major.points).shape[0], "isolated points removed")
    num_points_major = np.asarray(chunk_major.points).shape[0]

    print("Start of normalized Cuts")
    grouped_labels = normalized_cut(A, np.arange(num_points_major), T = 0.08)
    num_groups = len(grouped_labels)
    print("There are", num_groups, "cut regions")

    sorted_groups = sorted(grouped_labels, key=lambda x: len(x))
    num_points_top3 = np.sum([len(g) for g in sorted_groups[-3:]])
    top3_ratio = num_points_top3 / num_points_major
    print("Ratio of points in top 3 groups:", top3_ratio)

    random_colors = generate_random_colors(600)

    pcd_color = np.zeros((num_points_major, 3))

    for i, s in enumerate(grouped_labels):
        for j in s:
            pcd_color[j] = np.array(random_colors[i]) / 255

    pcd_chunk.paint_uniform_color([0, 0, 0])
    colors = kDTree_1NN_feature_reprojection(np.asarray(pcd_chunk.colors), pcd_chunk, pcd_color, chunk_major)
    pcd_chunk.colors = o3d.utility.Vector3dVector(colors)

    inliers = get_statistical_inlier_indices(pcd_ground_chunk)
    ground_inliers = get_subpcd(pcd_ground_chunk, inliers)
    mean_hight = np.mean(np.asarray(ground_inliers.points)[:,2])
    cut_hight = get_subpcd(ground_inliers, np.where(np.asarray(ground_inliers.points)[:,2] < (mean_hight + 0.2))[0])
    cut_hight.paint_uniform_color([0, 0, 0])

    merged_chunk = pcd_chunk + cut_hight

    index_file = str(center_id).zfill(6) + '.pcd'
    file = os.path.join("test_data", index_file)

    o3d.io.write_point_cloud(file, merged_chunk, write_ascii=False, compressed=False, print_progress=False)

    print("Pointcloud written to file")


Start of sequence  0
9619 points in downsampled chunk (major)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
There are 74 cut regions
Ratio of points in top 3 groups: 0.2504418338704647
Pointcloud written to file
Start of sequence  1
6468 points in downsampled chunk (major)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
There are 40 cut regions
Ratio of points in top 3 groups: 0.2911255411255411
Pointcloud written to file
Start of sequence  2
7564 points in downsampled chunk (major)
Adjacency Matrix built
0 isolated points removed
Start of normalized Cuts
There are 37 cut regions
Ratio of points in top 3 groups: 0.2513220518244315
Pointcloud written to file
Start of sequence  3
5933 points in downsampled chunk (major)


Now we can merge the chunks to one large Map!

In [ ]:
point_clouds = []

# List all files in the folder
files = os.listdir("test_data")
files.sort()

# Filter files with a .pcd extension
pcd_files = [file for file in files if file.endswith(".pcd")][:3]
print(pcd_files)
# Load each point cloud and append to the list
for pcd_file in pcd_files:
    file_path = os.path.join("test_data", pcd_file)
    point_cloud = o3d.io.read_point_cloud(file_path)
    point_clouds.append(point_cloud)

['000061.pcd', '000088.pcd', '000117.pcd']


In [8]:
merge = merge_chunks_unite_instances(point_clouds)
o3d.io.write_point_cloud("test_data/merge_part.pcd", merge, write_ascii=False, compressed=False, print_progress=False)

True

In [ ]:
DATASET_PATH = os.path.join('/media/cedric/Datasets1/semantic_kitti/')
SEQUENCE_NUM = 7


config_filtered = KittiOdometryDatasetConfig(
    cache=True,
    dataset_path=DATASET_PATH,
    sam_folder_name="sam_pred_medium",
    correct_scan_calibration=True,
    filters=FilterList(
        [
        ]
    ),
)

dataset = KittiOdometryDataset(config_filtered, SEQUENCE_NUM)

In [9]:
def kDTree_1NN_feature_reprojection(features_to, pcd_to, features_from, pcd_from, labels=None,max_radius=None, no_feature_label=[1,0,0]):
    '''
    Args:
        pcd_from: point cloud to be projected
        pcd_to: point cloud to be projected to
        search_method: search method ("radius", "knn")
        search_param: search parameter (radius or k)
    Returns:
        features_to: features projected on pcd_to
    '''
    from_tree = o3d.geometry.KDTreeFlann(pcd_from)
    labels_output = np.ones(np.asarray(pcd_to.points).shape[0],) * -1
    unique_colors = list(np.unique(np.asarray(pcd_from.colors),axis=0)) 
    
    for i, point in enumerate(np.asarray(pcd_to.points)):

        [_, idx, _] = from_tree.search_knn_vector_3d(point, 1)
        if max_radius is not None:
            if np.linalg.norm(point - np.asarray(pcd_from.points)[idx[0]]) > max_radius:
                features_to[i,:] = no_feature_label
                if labels is not None : 
                    labels[i] = -1
                    labels_output[i] = -1 
            else:
                features_to[i,:] = features_from[idx[0]]
                labels_output[i] = np.where((unique_colors == features_from[idx[0]]).all(axis=1))[0]
        else:
            features_to[i,:] = features_from[idx[0]]
            labels_output[i] = np.where((unique_colors == features_from[idx[0]]).all(axis=1))[0]
        
        
    if labels is not None : 
        return features_to,labels, labels_output
    else : 
        return features_to, None, labels_output

In [10]:
import copy
def color_pcd_by_labels(pcd, labels):
    
    colors = generate_random_colors(500)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colored.colors = o3d.utility.Vector3dVector(np.zeros(np.asarray(pcd.points).shape))
    
    unique_labels = list(np.unique(labels)) 
    for i in range(len(pcd_colored.points)):
        if labels[i] != (-1):
            color = colors[unique_labels.index(labels[i])]
            pcd_colored.colors[i] = np.array(color) / 255

    return pcd_colored

In [23]:
from open3d.pipelines import registration
import numpy as np 
merge = o3d.io.read_point_cloud("test_data/merge_part.pcd")
merge.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,max_nn=200))

for i in range(0,100):
	local_pcd = o3d.geometry.PointCloud()
	local_pcd.points = o3d.utility.Vector3dVector(dataset[i].point_cloud[:, :3])
	
	
	## label visualization 
	labeled_pcd = o3d.geometry.PointCloud()
	labeled_pcd.points = o3d.utility.Vector3dVector(dataset[i].point_cloud[:, :3])
	labeled_pcd.colors = o3d.utility.Vector3dVector(np.vstack([0,0,0] for i in range(np.asarray(labeled_pcd.points).shape[0])))
	panoptic_labels = dataset[i].panoptic_labels # semantics + panoptics combined 
	semantic_labels = dataset[i].semantic_labels
	instance_labels = dataset[i].instance_labels
	intensity = dataset[i].intensity
	
	
	transform = dataset.get_pose(i)
	
	
	
	local_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,max_nn=200))
	reg_p2l = registration.registration_icp(local_pcd, merge, 0.9, transform, registration.TransformationEstimationPointToPlane(), registration.ICPConvergenceCriteria(max_iteration=1000))
	transform = reg_p2l.transformation
	local_pcd.transform(transform)
	
	local_pcd.normals = o3d.utility.Vector3dVector([])
	
	local_pcd.paint_uniform_color([0, 0, 0])
	
	
	
	
	colors, labels,local_labels = kDTree_1NN_feature_reprojection(np.asarray(local_pcd.colors), local_pcd, np.asarray(merge.colors), merge,panoptic_labels, max_radius=0.2)
	labeled_pcd = color_pcd_by_labels(labeled_pcd,labels.reshape(-1,))
	local_pcd.colors = o3d.utility.Vector3dVector(colors)
	o3d.io.write_point_cloud("test_data/test.pcd", local_pcd, write_ascii=False, compressed=False, print_progress=False)
	
	
	
	#labeled_pcd.translate([0,120,0])
	#o3d.visualization.draw_geometries([local_pcd,labeled_pcd])
	unique_colors, labels = np.unique(colors, axis=0, return_inverse=True)
	unseen_mask = np.all(colors == [1,0,0], axis=1)
	labels[unseen_mask] = -1
	street_mask = np.all(colors == [0,0,0], axis=1)
	labels[street_mask] = 1
	np.savez('output_files/7_tarl/ ' + str(i) + '.npz',labels=labels)

	

	np.savez('output_files/7_original/' + str(i) + '.npz',ncut_labels=local_labels,kitti_labels=labels,pts=np.asarray(local_pcd.points),           
	intensities=intensity,kitti_labels_semantic=semantic_labels,kitti_labels_instance=instance_labels)




/tmp/ipykernel_443508/1041532388.py:14: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  labeled_pcd.colors = o3d.utility.Vector3dVector(np.vstack([0,0,0] for i in range(np.asarray(labeled_pcd.points).shape[0])))


KeyboardInterrupt: 

In [25]:
import numpy as np 
with np.load('output_files/7_original/0.npz') as data:
            xyz = data['pts'].astype(np.float)
            labels = data['ncut_labels'].astype(np.int32)  
            kitti_labels = data['kitti_labels']
            intensity = data['intensities']
            
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd = color_pcd_by_labels(pcd,labels)
o3d.visualization.draw_geometries([pcd])
        

/tmp/ipykernel_443508/69016850.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xyz = data['pts'].astype(np.float)
